# Flexopus Device Management template

## What is this?
In this Notebook, we created a simplified version of [Google's walktrough](https://colab.research.google.com/github/google/android-management-api-samples/blob/master/notebooks/quickstart.ipynb) on how to use the Android Device Management API.

If you are not familiar with Google Colaboratory (Colab), don't worry, it's really easy to use. A Colab Notebook consists of text and code blocks (or cells), written in Python programming language. Every block can be run using the play button on the left side of each of them. Click in a block to make it visible. Each code block has an output section right below the block where the code responses are printed.

The platform also provides so-called _forms_, which wrap over code block, allowing you to only see the importand input fields. We use this functionality to simplify the Notebook and focus only on what you will really use.

## How to get started?
This is a read-only document, so you will have to make a copy of it to save your changes.
1. Change to the **@gmail** account that you will use to provision the devices (top right corner, account selection).
2. Click on **Copy to Drive** on the top left part of the header. This will create a file named _Copy of Flexopus Device Management template.ipynb_ in you Google Drive's **Colab Notebooks** folder, and open it in Colab instantly.
3. Click the name on the top left corner to modify it to your preference.
4. Now you can start following the steps below.

### First run
When first using this notebook, make sure you follow **Steps 1-4**, running each block one after the other. Some of them requires some interaction, namely navigating to another url, and paste in some codes that the other web pages provide for you. Then run **Step 6** and **7** to create a policy and enroll devices. Please note that the order of execution matters!

### Enrolling multiple devices
The runtime environment preserves your variables while you are actively working. This means that to enroll multiple devices, one after the other, you will only need to run **Step 7** again, not all the previous steps.

### Coming back to this notebook
Make sure to work on a copy of this notebook in your Drive to preserve information. This way you can come back any time to enroll new devices, make changes to the policy, or delete devices (detach them from the enterprise). When opening the notebook again, please make sure to check that the entered `cloud_project_id` and `enterprise_name` are correct, and run **Steps 1**, **2**, and **4**.

To enroll a new device with **Step 7**, please make sure you run **Step 6** first, even if you make no changes to the policy.

In case you have lost your notebook, don't worry. Make another copy and:
* Retrieve your `cloud_project_id` from https://console.cloud.google.com/cloud-resource-manager
* Retrieve your `enterprise_name` from https://play.google.com/work/adminsettings, or run **Step 3** again, it will also give you back the previously created enterprise.
* If you do not remember your policy's name, list the created policies with **Step 5.1**

If you have any questions, do not hesitate to contact us.

In [ ]:
# @title # 1. Enter your project ID from Google Cloud platform { display-mode: "form" }
# @markdown If you do not have a Google Cloud Project yet, you can create one in the Google Cloud Console:
# @markdown 1. Go to the [Cloud Console](https://console.cloud.google.com/cloud-resource-manager).
# @markdown 2. Click CREATE PROJECT.
# @markdown 3. Enter your project details, and then click CREATE.
# @markdown 4. Take note of the project ID and paste it below, then run the cell.
cloud_project_id = '<paste-your-id-here>' # @param {type:"string"}

In [ ]:
# @title # 2. Authorize account { display-mode: "form" }
# @markdown Run this block to authorize Colab to access your devices. You will have to follow the printed url, sign in with your @gmail account, and copy the authorization code. Then close the new window and paste the code in the prompt field.
from apiclient.discovery import build
from google_auth_oauthlib.flow import Flow

# This is a public OAuth config that you can use to run this guide.
# However, use different credentials when building your own solution.
CLIENT_CONFIG = {
  'web': {
    'client_id':'882252295571-m8qpit609pj2hsql4j5k0bue1injrtd2.apps.googleusercontent.com',
    'auth_uri':'https://accounts.google.com/o/oauth2/auth',
    'token_uri':'https://oauth2.googleapis.com/token',
    'auth_provider_x509_cert_url':'https://www.googleapis.com/oauth2/v1/certs',
    'client_secret':'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'
  }
}
SCOPES = ['https://www.googleapis.com/auth/androidmanagement']
CALLBACK_URL = 'https://google.github.io/android-management-api-samples/oauth_callback.html'

# Run the OAuth flow.
flow = Flow.from_client_config(CLIENT_CONFIG, SCOPES)
flow.redirect_uri = CALLBACK_URL
auth_url, _ = flow.authorization_url()
print('Please visit this URL to authorize this application: {}'.format(auth_url))

code = input('Enter the authorization code: ')
flow.fetch_token(code=code)

# Create the API client.
androidmanagement = build('androidmanagement', 'v1', credentials=flow.credentials)

print('\nAuthentication succeeded.')

In [ ]:
# @title # 3. Create an enterprise (required only once) { display-mode: "form" }

# @markdown An **`Enterprise`** resource binds an organization to your Android Management solution. **`Devices`** and **`Policies`** both belong to an enterprise.

# @markdown To create an enterprise you need a **Gmail account** that's not already associated with an enterprise.

# @markdown To start the enterprise creation flow, **run this code block**.

# @markdown If you've already created an enterprise for this project, you can skip this step and enter your enterprise name in the next cell.

CALLBACK_URL = 'https://storage.googleapis.com/android-management-quick-start/enterprise_signup_callback.html'

# Generate a signup URL where the enterprise admin can signup with a Gmail
# account.
signup_url = androidmanagement.signupUrls().create(
    projectId=cloud_project_id,
    callbackUrl=CALLBACK_URL
).execute()

print('Please visit this URL to create an enterprise:', signup_url['url'])

enterprise_token = input('Enter the code: ')

# Complete the creation of the enterprise and retrieve the enterprise name.
enterprise = androidmanagement.enterprises().create(
    projectId=cloud_project_id,
    signupUrlName=signup_url['name'],
    enterpriseToken=enterprise_token,
    body={}
).execute()

enterprise_name = enterprise['name']

print('\nYour enterprise name is', enterprise_name)

In [ ]:
# @title # 4. Enter the enterprise name { display-mode: "form" }
enterprise_name = 'enterprises/<generated-enterprise-name>' # @param {type:"string"}


In [ ]:
# @title # 5.1. List policies { display-mode: "form" }
import pandas as pd
response = androidmanagement.enterprises().policies().list(parent=enterprise_name).execute()
pd.DataFrame(response['policies'])[['name', 'version']]

In [ ]:
# @title # 5.2. Execute policy command { display-mode: "form" }
import json
policy = "enterprises/<generated-enterprise-name>/policies/flexopus" # @param {type:"string"}
command = "get information" # @param ["get information", "delete policy"]
if command == "get information":
  response = androidmanagement.enterprises().policies().get(name=policy).execute()
if command == "delete policy":
  response = androidmanagement.enterprises().policies().delete(name=policy).execute()

print(json.dumps(response, sort_keys=True, indent=4))

In [ ]:
# @title # 6. Create or update your policy { display-mode: "form" }
# @markdown <font color="orange">
# @markdown ⚠️ WARNING:
# @markdown Due to a know issue in the Google Management API, the provisioning
# @markdown may stop at "Installing work apps", especially when setting up PoE
# @markdown devices. If you experience this, change install type from
# @markdown <code>KIOSK</code> to <code>AVAILABLE</code>. Once you have set up
# @markdown the devices, you can update the policy to use  <code>KIOSK</code>
# @markdown mode again.
# @markdown </font>
policy_name = "flexopus" # @param {type:"string"}
install_type = "KIOSK" # @param ["KIOSK", "AVAILABLE"]
policy_content = {
  "applications": [
    {
      "packageName": "com.flexopus.deviceconnect",
      "installType": install_type,
      "defaultPermissionPolicy": "GRANT",
      "autoUpdateMode": "AUTO_UPDATE_HIGH_PRIORITY"
    }
  ],
  "keyguardDisabled": True,
  "systemUpdate": {
    "type": "WINDOWED",
    "startMinutes": 120,
    "endMinutes": 240
  }
}
policy_name = enterprise_name + '/policies/' + policy_name

androidmanagement.enterprises().policies().patch(
    name=policy_name,
    body=policy_content
).execute()

In [ ]:
# @title # 7. Generate an enrollment token { display-mode: "form" }
# @markdown Generate a new enrollment token using the previously set `enterprise_name` and `policy_name`. Make sure you have run Step 6. before.
enrollment_token = androidmanagement.enterprises().enrollmentTokens().create(
    parent=enterprise_name,
    body={"policyName": policy_name}
).execute()

print('Instead of email, enter: afw#setup')
print()

split_token = ' '.join([enrollment_token['value'][i:i+4] for i in range(0, len(enrollment_token['value']), 4)])
print('Enrollment token:       ', split_token, '(without spaces)')
print()
print('Valid until:            ', enrollment_token['expirationTimestamp'])

In [ ]:
# @title # 8.1 List devices { display-mode: "form" }
response = androidmanagement.enterprises().devices().list(parent=enterprise_name).execute()
pd.DataFrame(response['devices'])[[
  'name', 'state', 'appliedState', 'policyCompliant', 'enrollmentTime', 'lastStatusReportTime', 'appliedPolicyVersion'
]]

In [ ]:
# @title # 8.2. Execute device command { display-mode: "form" }
device_name = "enterprises/<generated-enterprise-name>/devices/<device-id>" # @param {type:"string"}
command = "get information" # @param ["get information", "delete device"]
if command == "get information":
  response = androidmanagement.enterprises().devices().get(name=device_name).execute()
if command == "delete device":
  response = androidmanagement.enterprises().devices().delete(name=device_name).execute()

print(json.dumps(response, sort_keys=True, indent=4))